In [1]:
import random
import os
import re
from glob import glob
from tqdm import tqdm
import numpy as np

import pandas as pd
import ast
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from PIL import Image, ImageDraw, _imaging
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras import metrics
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from sklearn.model_selection import train_test_split
import tensorflow as tf
from datetime import datetime
from keras.callbacks import History, ModelCheckpoint
history = History()
from keras.models import model_from_json

Using TensorFlow backend.


In [6]:
# def generateTrainData(output_height, output_width, recordsPerTrainClass, skiprows=0):

#     test_df = pd.read_csv('test_simplified.csv')

#     # Get names of all train csv files with the pattern match below
#     fnames = glob('train_simplified/*.csv')

#     rows = recordsPerTrainClass
#     # Get n rows from all the csv files and append them into one dataframe
#     train_df = pd.DataFrame(columns=pd.read_csv(fnames[0], nrows=1).columns)
#     for name in fnames:
#         if skiprows == 0:
#             data = pd.read_csv(name, nrows=recordsPerTrainClass)
#         else:
#             data = pd.read_csv(name, nrows=recordsPerTrainClass, skiprows=range(1,skiprows))
#         train_df = train_df.append(data)

# #     print(train_df.shape)
        
#     train_df = train_df.reset_index().drop('index', axis=1)
#     # Get only those which were correctly recognized
#     train_df = train_df[train_df['recognized'] == True]
    
#     # Convert the drawing column to matrix
# #     train_df['drawing'] = train_df['drawing'].apply(ast.literal_eval)
# #     test_df['drawing'] = test_df['drawing'].apply(ast.literal_eval)
#     train_df['drawing'] = train_df['drawing'].apply(eval)
#     test_df['drawing'] = test_df['drawing'].apply(eval)

#     # Convert drawing to images
#     train_df['img'] = train_df['drawing'].apply(lambda x: draw_it(x, output_height, output_width))
#     test_df['img'] = test_df['drawing'].apply(lambda x: draw_it(x, output_height, output_width))
#     # train_df['img'] = train_df[['drawing']].apply(lambda x: draw_it(x['drawing'], output_height, output_width), axis=1)
#     # test_df['img'] = test_df[['drawing']].apply(lambda x: draw_it(x['drawing'], output_height, output_width), axis=1)
    
#     return train_df, test_df
    
def showSampleImgs():
    n_samp = 3
    train_df_sample = train_df.sample(n_samp)
    plt.figure(figsize=(4,4))
    for i in range(n_samp):
        draw = train_df_sample.iloc[i]['drawing']
        label = train_df_sample.iloc[i]['word']
        plt.subplot(n_samp,1,i+1)
        for stroke in draw:
            plt.plot(stroke[0], stroke[1], marker='.', color='black')
            plt.title(label)
            plt.axis('off')
    plt.show()    
    
# # Convert drawings to images
# def draw_it(raw_strokes, output_height, output_width):
#     image = Image.new("P", (255,255)
# #                       , color=1
#             )
#     image_draw = ImageDraw.Draw(image)

#     for stroke in raw_strokes:
#         for i in range(len(stroke[0])-1):

#             image_draw.line([stroke[0][i], 
#                              stroke[1][i],
#                              stroke[0][i+1], 
#                              stroke[1][i+1]],
#                             fill=255, width=6)
#     # Reduce image size
#     image = image.resize((output_height,output_width),Image.ANTIALIAS)
    
#     return np.array(image)

# Show an image from the dataframe
def showImgFromDf(df, index):
    # Show an image
    plt.imshow(df.iloc[index]['img'],cmap='gray')
    plt.title(df.iloc[index]['word'])
    plt.show()
    
# def CNN_dataPrep(train_df, img_height, img_width):
    
#     num_classes = train_df['word'].nunique()
#     # Data Preprocessing

#     # Shuffle the data
#     random.seed(111)
#     train_df = train_df.sample(train_df.shape[0])
    
#     # input image dimensions
#     img_rows, img_cols = img_height, img_width
#     input_shape = (img_rows, img_cols)
    
#     # Reshape the array
#     imgArr = np.vstack(train_df['img'].values).flatten().reshape((train_df['img'].shape[0], img_rows, img_cols))
#     imgArr_test = np.vstack(test_df['img'].values).flatten().reshape((test_df['img'].shape[0], img_rows, img_cols))
    
#     # In Keras, the layers used for two-dimensional convolutions expect pixel values with the dimensions [width][height][pixels] for TF.
#     # In the case of RGB, the first dimension pixels would be 3 for the red, green and blue components and it would be like having 3 image inputs for every color image. In this case, the pixel values are gray scale, the pixel dimension is set to 1.
#     imgArr = imgArr.reshape(imgArr.shape[0], img_rows, img_cols, 1).astype('float32')
#     imgArr_test = imgArr_test.reshape(imgArr_test.shape[0], img_rows, img_cols, 1).astype('float32')
    
#     # Initialize the y_train
#     y_train = train_df['word']
    
#     # Convert class labels from categorical to numerical
#     unique_classes_list = y_train.unique()
#     map_class_to_numeric = {k: v for v, k in enumerate(y_train.unique())}
#     map_numeric_to_class = {v: k for k, v in map_class_to_numeric.iteritems()}
#     y_train_numeric = y_train.apply(lambda x: map_class_to_numeric[x])

#     # convert class vectors to binary class matrices
#     y_train_one_hot = keras.utils.to_categorical(y_train_numeric, num_classes)
#     num_classes = y_train_one_hot.shape[1]
    
#     X_train, X_test, y_train, y_test = train_test_split(imgArr, y_train_one_hot, test_size=0.2)
    
#     return X_train, X_test, y_train, y_test, imgArr_test, map_class_to_numeric, map_numeric_to_class
    
    
def top_3_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)


def baseline_conv_model(num_filters, num_classes, img_rows, img_cols):
    model = Sequential()
    model.add(Conv2D(num_filters, (5,5), input_shape=(img_rows,img_cols,1), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
#     model.add(Conv2D(num_filters*2, (5,5), input_shape=(img_rows,img_cols,1), activation='relu')) 
#     model.add(MaxPooling2D(pool_size=(2,2)))
#     model.add(Dropout(0.2))
#     model.add(Conv2D(num_filters*2, (3,3), input_shape=(img_rows,img_cols,1), activation='relu')) 
#     # model.add(MaxPooling2D(pool_size=(2,2)))
#     model.add(Dropout(0.2))
    model.add(Flatten())
#     model.add(Dense(1000, activation='relu'))
#     model.add(Dropout(0.2))
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.2))
#     model.add(Dense(num_classes, activation='softmax'))
    model.add(Dense(units=num_classes))
#     model.add(Activation('softmax'))
    model.add(Activation(tf.nn.softmax))

    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', top_3_accuracy, 'categorical_crossentropy'])
    return model


In [105]:
def draw_it(raw_strokes, output_height, output_width):
    raw_strokes = eval(raw_strokes)
    image = Image.new("P", (255,255)
#                       , color=1
            )
    image_draw = ImageDraw.Draw(image)

    for stroke in raw_strokes:
        for i in range(len(stroke[0])-1):

            image_draw.line([stroke[0][i], 
                             stroke[1][i],
                             stroke[0][i+1], 
                             stroke[1][i+1]],
                            fill=255, width=6)
    # Reduce image size
    image = image.resize((output_height,output_width),Image.ANTIALIAS)
    
    return np.array(image)


class DataGenerator(keras.utils.Sequence):
    def __init__(self, num_samples_to_train, batch_size, img_width, img_height):
        self.batch_size = batch_size
        self.num_samples_to_train = num_samples_to_train
        self.img_width = img_width
        self.img_height = img_height
    
    def __len__(self):
        return np.ceil(self.num_samples_to_train / float(self.batch_size))
    
    def __getitem__(self, idx):
        
        X, y = self._getNextData(idx)
#         pp.append(X)
        
        return X, y

    
    def _getNextData(self, idx):
#         TO DO: SAMPLE THE DF
        fnames = glob('train_simplified/*.csv')
        numClasses = len(fnames)

        numRowsEachDf = int(self.batch_size / float(numClasses))
        skipRows = (idx * numRowsEachDf)
        print("(" + str(skipRows) + "," + str(numRowsEachDf) + ")")
        
        # Get n rows from all the csv files and append them into one dataframe
        train_df = pd.DataFrame(columns=pd.read_csv(fnames[0], nrows=1).columns)
        
        for name in fnames:
            if skipRows == 0:
                data = pd.read_csv(name, nrows=numRowsEachDf)
            else:
                data = pd.read_csv(name, nrows=numRowsEachDf, skiprows=range(1,skipRows)) # 0th row is the header, so not skipping that
            # Append the training data of all the classes
            train_df = train_df.append(data)
            
        train_df = train_df.reset_index().drop('index', axis=1)

        # Shuffle all data points
        train_df = train_df.sample(train_df.shape[0])
        
        # Get only those which were correctly recognized
        train_df = train_df[train_df['recognized'] == True]

        # Convert drawing to images
        train_df['drawing'] = train_df['drawing'].apply(lambda x: draw_it(x, self.img_height, self.img_width))

            # Reshape the array
        train_imgArr = np.vstack(train_df['drawing'].values).flatten().reshape((train_df['drawing'].shape[0], self.img_height, self.img_width))
    
        # In Keras, the layers used for two-dimensional convolutions expect pixel values with the dimensions [width][height][pixels] for TF.
        # In the case of RGB, the first dimension pixels would be 3 for the red, green and blue components and it would be like having 3 image inputs for every color image. In this case, the pixel values are gray scale, the pixel dimension is set to 1.
        train_X = train_imgArr.reshape(train_imgArr.shape[0], self.img_height, self.img_width, 1).astype('float32')
        
        
         # Initialize the y_train
        y_train = train_df['word']
    
        # Convert class labels from categorical to numerical
        unique_classes_list = y_train.unique()
        map_class_to_numeric = {k: v for v, k in enumerate(unique_classes_list)}
        map_numeric_to_class = {v: k for k, v in map_class_to_numeric.iteritems()}
        y_train_numeric = y_train.apply(lambda x: map_class_to_numeric[x])

        # convert class vectors to binary class matrices
        train_y = keras.utils.to_categorical(y_train_numeric, numClasses)

        return train_X, train_y
    

In [103]:
pp = []
dataGenerator = DataGenerator(num_samples_to_train=5000, batch_size=1000, img_width=32, img_height=32)


In [104]:
model = baseline_conv_model(4, 340, 32, 32)
model.fit_generator(
        generator=dataGenerator,
        epochs=1,
        verbose=1
)
#       steps_per_epoch=(num_training_samples // batch_size),
#       validation_data=my_validation_batch_generator,
#       validation_steps=(num_validation_samples // batch_size),
#       use_multiprocessing=False,
#       workers=16,
#       max_queue_size=32)


Epoch 1/1
(0,2)


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(2,2)
5/5 [==============================] - 14s 3s/step - loss: 16.0284 - acc: 0.0026 - top_3_accuracy: 0.0287 - categorical_crossentropy: 16.0284


In [98]:
pp[2]

,countrycode,drawing,key_id,recognized,timestamp,word
389,US,"[[[109, 102, 101, 105, 128, 154, 168, 177, 210...",4739868104065024,True,2017-01-27 16:26:18.424590,dragon
51,US,"[[[0, 0, 9, 13], [0, 92, 150, 210]], [[0, 63, ...",6290332788457472,True,2017-03-27 05:14:02.457680,skyscraper
608,US,"[[[81, 65, 54, 28, 3, 0, 5, 20, 39, 74, 125, 1...",6636873587884032,True,2017-01-23 00:46:51.549530,fan
319,SA,"[[[205, 194, 190, 192, 183, 175, 176, 154, 149...",5835957108473856,False,2017-03-12 10:49:29.483170,tooth
524,US,"[[[2, 0, 3, 22, 33, 132, 188, 224, 240, 255, 2...",5736884426768384,True,2017-03-17 12:59:36.106880,microwave
504,AU,"[[[53, 105, 181], [18, 6, 0]], [[48, 52, 50, 0...",4552316948578304,True,2017-01-27 08:57:02.068470,stop sign
416,CH,"[[[99, 84, 80], [3, 25, 56]], [[93, 103, 116, ...",6746256573464576,True,2017-01-23 21:23:00.117470,octopus
520,US,"[[[27, 0, 7, 40, 47, 20], [0, 41, 74, 73, 41, ...",4836123148812288,True,2017-03-06 20:00:22.521560,ant
662,KW,"[[[2, 0, 13, 17, 27, 43, 67, 136, 212, 255, 25...",5346849391116288,True,2017-01-24 14:45:40.925790,envelope
454,PL,"[[[170, 186, 193, 194, 198, 217, 255, 250, 231...",5771933759045632,True,2017-03-20 11:46:37.832130,swan


In [131]:
# %%time 
# recordsPerTrainClass = 4
# skiprows = 0
# img_height = img_width = 32

# # Get data and transform drawing into image
# train_df, test_df = generateTrainData(img_height, img_width, recordsPerTrainClass, skiprows)


In [132]:
# %%time
# # Transform data for CNN
# X_train, X_test, y_train, y_test, imgArr_test, map_class_to_numeric, map_numeric_to_class = CNN_dataPrep(train_df, img_height, img_width)

In [21]:
num_filters = 2
num_classes = 340
continueTrain = False
if continueTrain == False:
    model = baseline_conv_model(num_filters, num_classes, img_height, img_width)

# checkpoint
filepath="Saved_Models/weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]

print(model.summary())

hist = model.fit(X_train, 
          y_train, 
          validation_data=(X_test, y_test),
          epochs=2, 
          batch_size=5, 
          verbose=2,
          callbacks = callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 4)         104       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 4)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 4)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               392500    
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 340)               170340    
__________

In [67]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
# model

CNN Error: 99.71%


In [18]:
# Make predictions
preds = model.predict(imgArr_test)
outputDf = test_df.copy()[['key_id']]
outputDf['word'] = ''
map_numeric_to_class_space_normal = map_numeric_to_class.copy()
for key in map_numeric_to_class_space_normal:
    map_numeric_to_class_space_normal[key] = (map_numeric_to_class_space_normal[key].replace(" ", "_"))

for i in tqdm(range(preds.shape[0])):
    outputDf['word'].at[i] = ' '.join(([map_numeric_to_class_space_normal[predClass] for predClass in [tup[1] for tup in sorted(zip(preds[i], range(340)), reverse=True)[:3]]]))

# Create csv
outputDf.to_csv('initial_pred.csv', index=False)

100%|██████████| 112199/112199 [00:22<00:00, 4887.68it/s]


In [21]:
# Save the model
# serialize model to JSON
model_json = model.to_json()
with open(("Saved_Models/model_" + str(datetime.today())[:16].replace(" ", "_").replace(":","-") + ".json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(("Saved_Models/model_" + str(datetime.today())[:16].replace(" ", "_").replace(":","-") + ".h5"))
print("Saved model to disk")

Saved model to disk


In [25]:
# model = ""

In [66]:
# Load the model back
# load json and create model
json_file = open('Saved_Models/model_2018-11-09_22-07.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("Saved_Models/model_2018-11-09_22-07.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', top_3_accuracy, 'categorical_crossentropy'])
# score = loaded_model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
model = loaded_model

Loaded model from disk
